In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from keras.applications.vgg16 import preprocess_input, decode_predictions
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
IMAGE_SIZE = [224, 224]
train_path = 'C://Users/KIIT//Desktop//Lupita//Train_set//'
valid_path = 'C://Users/KIIT//Desktop//Lupita//Test_set//'

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False
  

folders = glob(train_path)
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()
model.compile(
  loss= 'binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('C://Users//KIIT//Desktop//Lupita//Train_set//',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('C://Users//KIIT//Desktop//Lupita//Test_set//',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=7,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


model.save('C://Users//KIIT//Desktop//Lupita//maskmodel.h5')


In [ ]:
from PIL import Image
from keras.applications.vgg19 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
model = load_model('C://Users//KIIT/Desktop//Lupita//maskmodel.h5')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    faces=face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=5)
    
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face
video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
        img_array = np.array(im)
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        print(pred)                   
        name="None matching"
        
        if(pred[0][0]==0.):
            name='Mask On'
            cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        else:
            name='Mask Not On'
            cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (139,0,0), 2)              
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(2)==27:
                break    
video_capture.release()
cv2.destroyAllWindows()

       

